In [ ]:
import os
import comet
import sacrebleu
import pandas as pd

In [ ]:
translations_path = "translations/mistral7b_v1_translations.csv"
evaluation_path = "evaluations/mistral7b_v1_evaluation.csv"
comet_dir = "./comet"
comet_model_name = "wmt20-comet-da"

In [ ]:
df = pd.read_csv(translations_path)
sources = df["sources"].tolist()
references = df["references"].tolist()
translations = df["translations"].tolist()

In [ ]:
# Calculate BLEU
bleu = sacrebleu.corpus_bleu(translations, [references])  # for spBLEU: tokenize='flores200'
bleu = round(bleu.score, 2)
print("BLEU:", bleu)

# Calculate chrF++
chrf = sacrebleu.corpus_chrf(translations, [references], word_order=2)  # for chrF++ word_order=2
chrf = round(chrf.score, 2)
print("chrF++:", chrf)

# Calculate TER
metric = sacrebleu.metrics.TER()
ter = metric.corpus_score(translations, [references])
ter = round(ter.score, 2)
print("TER:", ter)

In [ ]:
# Calculate COMET
df = pd.DataFrame({"src":sources, "mt":translations, "ref":references})
data = df.to_dict('records')

model_path = os.path.join(comet_dir, comet_model_name)
if not os.path.exists(model_path):
    model_path = comet.download_model(model=comet_model_name, saving_directory=comet_dir)
model = comet.load_from_checkpoint(model_path)

seg_scores, sys_score = model.predict(data, batch_size=128, gpus=1).values()
comet = round(sys_score*100, 2)
print("COMET:", comet)

In [ ]:
folder = os.path.dirname(evaluation_path)
os.makedirs(folder, exist_ok=True)

df = pd.DataFrame({"BLEU": [bleu],
                   "chrF++": [chrf],
                   "TER": [ter],
                   "COMET": [comet]},
                  )

df.to_csv(evaluation_path, index=False)
df.head()